# Homework 2

In [1]:
# import dataset
import pandas as pd
spy = pd.read_csv('SPY_HW2.csv', index_col='Date', parse_dates=True)
irx = pd.read_csv('IRX_HW2.csv', index_col='Date', parse_dates=True)
oakmx = pd.read_csv('OAKMX_HW2.csv', index_col='Date', parse_dates=True)
arkk_m = pd.read_csv('ARKK_Montlhly_HW2.csv', index_col='Date', parse_dates=True)
FFM = pd.read_csv('FamaFrenchMonthly_HW2.csv', index_col='Date', parse_dates=True)

## Problem 1

In [2]:
import numpy as np
spy_df = spy.copy()
# Calculate daily close-to-close log returns for SPY using adjusted close prices
spy_df['CtoC_Adj'] = np.log(spy_df['Adj Close'] / spy_df['Adj Close'].shift(1))
spy_df = spy_df.dropna()
# Calculate average daily close-to-close log return for SPY for full sample period
avg_spy_return = spy_df['CtoC_Adj'].mean()
# Calculate average daily close-to-close log return for SPY for the first trading day of each month
first_days_idx = spy_df.groupby(spy_df.index.to_period("M")).head(1).index
avg_spy_return_first_day = spy_df.loc[first_days_idx, "CtoC_Adj"].mean()
# Calculate average daily close-to-close log return for SPY for the third-to-the-last trading day of each month

# 每月倒数第3个交易日（要求该月有>=3个交易日）
# 这里扣了三分，因为不能直接用nth(-3)，会包含交易日少于3天的月份
# m = spy_df.index.to_period('M')
# sizes = spy_df.groupby(m)['Adj Close'].transform('size')
# pos_from_end = spy_df.groupby(m, sort=False).cumcount(ascending=False)  # 月末=0，倒2=1，倒3=2
# third_last_idx = spy_df.index[(sizes >= 3) & (pos_from_end == 2)]
third_last_idx = spy_df.groupby(spy_df.index.to_period("M")).nth(-3).index
avg_spy_third_last_day = spy_df.loc[third_last_idx, "CtoC_Adj"].mean()
# Display results
print(f"Average Daily Close-to-Close Log Return for SPY (Full Sample): {avg_spy_return * 100:.3f}")
print(f"Average Daily Close-to-Close Log Return for SPY (First Trading Day of Each Month): {avg_spy_return_first_day * 100:.3f}")
print(f"Average Daily Close-to-Close Log Return for SPY (Third-to-the-Last Trading Day of Each Month): {avg_spy_third_last_day * 100:.3f}")

Average Daily Close-to-Close Log Return for SPY (Full Sample): 0.040
Average Daily Close-to-Close Log Return for SPY (First Trading Day of Each Month): 0.211
Average Daily Close-to-Close Log Return for SPY (Third-to-the-Last Trading Day of Each Month): 0.067


## <mark> Problem solution summary: <mark>

For the above questions in **problem 1**, we load the data from the spy CSV files into pandas dataframes. We then compute the daily close-to-close log returns using the adjusted close prices by applying the numpy log function. We calculate the average daily return for the full sample period, as well as for specific trading days (first trading day and third-to-the-last trading day of each month) by grouping the data by month and using the head and nth functions. Finally, we display the results formatted to three decimal places. For computing the average for specific days in each month, we need to use groupby function to group the data by month and select the relevant indexes.

## Problem 2

In [3]:
# Load data
spy_df = spy.copy()
irx_df = irx.copy()
# add _ to column names to avoid confusion after merging
spy_df.columns = ['Adj Close_SPY', 'Close_SPY', 'High_SPY', 'Low_SPY', 'Open_SPY', 'Volume_SPY']
irx_df.columns = ['Adj Close_IRX', 'Close_IRX', 'High_IRX', 'Low_IRX', 'Open_IRX', 'Volume_IRX']
# Merge data on the Date index
df_merge = spy_df.join(irx_df['Close_IRX'], how="left")
# Compute T-bill returns, using calendar days between observations
df_merge['T-bill Return'] = (df_merge['Close_IRX'].shift(1) / 36500) * df_merge.index.to_series().diff().dt.days
# compute excess returns for SPY
df_merge['CtoC_Adj'] = df_merge['Adj Close_SPY'].pct_change()
df_merge['Excess Return_SPY'] = df_merge['CtoC_Adj'] - df_merge['T-bill Return']
# Drop rows with NaN values
df_merge = df_merge.dropna(subset = ['Excess Return_SPY'])
# Compute the mean of excess return for SPY
avg_excess_return_spy = df_merge['Excess Return_SPY'].mean()
# Compute the standard deviation of excess return for SPY
std_excess_return_spy = df_merge['Excess Return_SPY'].std()
# Display results
print(f"Average Excess Return for SPY: {avg_excess_return_spy * 100:.3f}")
print(f"Standard Deviation of Excess Return for SPY: {std_excess_return_spy * 100:.3f}")

Average Excess Return for SPY: 0.037
Standard Deviation of Excess Return for SPY: 1.178


## <mark> Problem solution summary: <mark>

From the above questions in **problem 2**, we load the SPY and IRX data from the CSV files into pandas dataframes. We then merge the two dataframes on the Date index to align the data. We compute the T-bill returns using the previous day's closing price of IRX and the number of calendar days between observations. Next, we calculate the excess returns for SPY by subtracting the T-bill returns from the daily close-to-close simple returns of SPY (For matching two forms of return). Finally, we compute and display the average and standard deviation of the excess returns for SPY.

## Problem 3

In [4]:
# Load data
oakmx_df = oakmx.copy()
# Compute the maximun drawdown for OAKMX
oakmx_df['CtoC_Adj'] = oakmx_df['Adj Close'].pct_change()
oakmx_df['Cumulative Return'] = (1 + oakmx_df['CtoC_Adj']).cumprod()
oakmx_df['Cumulative Max'] = oakmx_df['Cumulative Return'].cummax()
oakmx_df['Drawdown'] = oakmx_df['Cumulative Return'] / oakmx_df['Cumulative Max'] - 1
max_drawdown = oakmx_df['Drawdown'].min()
# Display the maximum drawdown
print(f"Maximum Drawdown for OAKMX: {max_drawdown * 100:.3f}")

Maximum Drawdown for OAKMX: -56.195


## <mark> Problem solution summary: <mark>

From the above questions in **problem 3**, we load the OAKMX data from the CSV file into a pandas dataframe. We then compute the daily close-to-close simple returns using the adjusted close prices. Next, we calculate the cumulative returns by taking the cumulative product of (1 + daily returns). We also compute the cumulative maximum of the cumulative returns to identify the peak values. The drawdown is then calculated as the percentage decline from the cumulative maximum to the current cumulative return. Finally, we find and display the maximum drawdown, which is the minimum value of the drawdown series.

## Problem 4

In [5]:
# Load data
oakmx_df = oakmx.copy()
# Merge data with irx on the Date index
df_merge = oakmx_df.join(irx_df['Close_IRX'], how="inner")
# Compute T-bill returns, using calendar days between observations
df_merge['T-bill Return'] = (df_merge['Close_IRX'].shift(1) / 36500) * df_merge.index.to_series().diff().dt.days
# Compute excess returns for OAKMX
df_merge['CtoC_Adj'] = df_merge['Adj Close'].pct_change()
df_merge['Excess Return_OAKMX'] = df_merge['CtoC_Adj'] - df_merge['T-bill Return']
# Calculate downside deviation of excess return for OAKMX
downside_returns = np.minimum(df_merge['Excess Return_OAKMX'], 0)
downside_deviation = np.sqrt((downside_returns**2).mean())

# Display results
print(f"Downside Deviation of Excess Return for OAKMX: {downside_deviation * 100:.3f}")

Downside Deviation of Excess Return for OAKMX: 0.803


## <mark> Problem solution summary: <mark>

From the above questions in **problem 4**, we load the OAKMX and IRX data from the CSV files into pandas dataframes. We then merge the two dataframes on the Date index to align the data. We compute the T-bill returns using the previous day's closing price of IRX and the number of calendar days between observations. Next, we calculate the excess returns for OAKMX by subtracting the T-bill returns from the daily close-to-close simple returns of OAKMX. Finally, we compute the downside deviation of the excess returns by taking the square root of the mean of squared negative excess returns and display the result.

## Problem 5

In [6]:
# Load data
oakmx_df = oakmx.copy()
# Merge data on the Date index
df_merge = oakmx_df.join(irx_df['Close_IRX'], how="inner")
df_merge['MA200'] = df_merge['Adj Close'].rolling(window=200, min_periods=200).mean()
# Compute T-bill returns, using calendar days between observations
df_merge['T-bill Return'] = (df_merge['Close_IRX'].shift(1) / 36500) * df_merge.index.to_series().diff().dt.days
# Compute excess returns for OAKMX
df_merge['CtoC_Adj'] = df_merge['Adj Close'].pct_change()
df_merge['Excess Return_OAKMX'] = df_merge['CtoC_Adj'] - df_merge['T-bill Return']
# Drop rows with NaN values
df_merge = df_merge.dropna(subset = ['Adj Close', 'Excess Return_OAKMX', 'MA200'])

In [7]:
# Separate excess returns based on whether the previous day's adjusted close price is above or below the 200-day moving average
above = df_merge.loc[df_merge['Adj Close'].shift(1) > df_merge['MA200'].shift(1), 'Excess Return_OAKMX']
below = df_merge.loc[df_merge['Adj Close'].shift(1) < df_merge['MA200'].shift(1), 'Excess Return_OAKMX']

# Function to calculate mean and t-statistic
def calc_stats(series):
    mean = series.mean()
    std = series.std()
    n = len(series)
    tstat = (mean / std) * np.sqrt(n) if n > 1 and std > 0 else np.nan
    return mean, tstat
# Calculate statistics for both groups
mean_above, tstat_above = calc_stats(above)
mean_below, tstat_below = calc_stats(below)
# Significance level
alpha = 0.05
# Display results
print(f"Mean Excess Return on OAKMX (Above 200-day MA): {mean_above * 100:.3f}, t-statistic: {tstat_above:.3f}")
# Check if the means are statistically significant at the 5% level (two-tailed test)
if abs(tstat_above) > 1.96:
    print(f"The mean excess return on OAKMX when above the 200-day MA is statistically significant at the {alpha*100}% level.")
else:
    print(f"The mean excess return on OAKMX when above the 200-day MA is not statistically significant at the {alpha*100}% level.")

print(f"Mean Excess Return on OAKMX (Below 200-day MA): {mean_below * 100:.3f}, t-statistic: {tstat_below:.3f}")
if abs(tstat_below) > 1.96:
    print(f"The mean excess return on OAKMX when below the 200-day MA is statistically significant at the {alpha*100}% level.")
else:
    print(f"The mean excess return on OAKMX when below the 200-day MA is not statistically significant at the {alpha*100}% level.")

Mean Excess Return on OAKMX (Above 200-day MA): 0.043, t-statistic: 4.219
The mean excess return on OAKMX when above the 200-day MA is statistically significant at the 5.0% level.
Mean Excess Return on OAKMX (Below 200-day MA): 0.034, t-statistic: 0.791
The mean excess return on OAKMX when below the 200-day MA is not statistically significant at the 5.0% level.


## <mark> Problem solution summary: <mark>

From the above questions in **problem 5**, we load the OAKMX and IRX data from the CSV files into pandas dataframes. We then merge the two dataframes on the Date index to align the data. We compute the 200-day moving average of the adjusted close prices for OAKMX. Next, we calculate the T-bill returns using the previous day's closing price of IRX and the number of calendar days between observations. We then compute the excess returns for OAKMX by subtracting the T-bill returns from the daily close-to-close simple returns of OAKMX. We calculate the mean excess returns for periods when the previous adjusted close price is above and below the 200-day moving average. Finally, we perform a t-test to determine if these means are statistically significant at the 5% level and display the results.

## Problem 6

In [8]:
# load data
arkk_m_df = arkk_m.copy()
FFM_df = FFM.copy()
# Construct a DataFrame of monthly return of ARKK over the months from June 2018 to June 2023
arkk_m_df = arkk_m_df[(arkk_m_df.index >= '2018-06-01') & (arkk_m_df.index <= '2023-06-30')]
# Compute monthly close-to-close simple returns for ARKK using adjusted close prices (Matching Fama-French factors)
arkk_m_df['CtoC_Adj'] = arkk_m_df['Adj Close'].pct_change()
arkk_m_df = arkk_m_df.dropna()
# arkk_m_df
# Merge ARKK returns with Fama-French factors
df_merge = arkk_m_df.join(FFM_df, how="inner")
df_merge

,Adj Close,Close,High,Low,Open,Volume,CtoC_Adj,Mkt-RF,SMB,HML,RF
Date,,,,,,,,,,,
2018-07-01,41.632938,44.549999,47.950001,43.509998,44.779999,6615900,-0.009560,3.20,-2.23,0.49,0.16
2018-08-01,46.445724,49.700001,49.869999,44.349998,44.650002,7169900,0.115600,3.45,1.14,-3.91,0.16
2018-09-01,44.240253,47.340000,49.500000,46.180000,49.430000,5289000,-0.047485,0.06,-2.29,-1.62,0.15
2018-10-01,39.791931,42.580002,48.480000,39.389999,48.349998,9737200,-0.100549,-7.65,-4.70,3.38,0.19
2018-11-01,41.539486,44.450001,46.220001,39.889999,43.180000,5897600,0.043917,1.71,-0.73,0.36,0.18
2018-12-01,34.754856,37.189999,46.000000,34.750000,46.000000,9025700,-0.163330,-9.55,-2.31,-1.92,0.19
2019-01-01,41.523792,43.049999,43.240002,35.910000,36.270000,7700100,0.194762,8.37,2.76,-0.39,0.21
2019-02-01,44.909355,46.560001,46.840000,42.125000,43.009998,5947500,0.081533,3.42,2.05,-2.66,0.18
2019-03-01,45.073334,46.730000,48.150002,43.380001,46.849998,8062800,0.003651,1.10,-3.03,-4.14,0.19


### Problem 6.a

In [9]:
df_merge['RF'] = df_merge['RF'] / 100
df_merge['Excess Return_ARKK'] = df_merge['CtoC_Adj'] - df_merge['RF']
# Compute the average annual return, average annual excess return and annual standard deviation
avg_annual_return_arkk = df_merge['CtoC_Adj'].mean() * 12
avg_annual_excess_return_arkk = df_merge['Excess Return_ARKK'].mean() * 12
annual_std_arkk = df_merge['CtoC_Adj'].std() * np.sqrt(12)
# Display results
print(f"Average Annual Return for ARKK: {avg_annual_return_arkk * 100:.3f}")
print(f"Average Annual Excess Return for ARKK: {avg_annual_excess_return_arkk * 100:.3f}")
print(f"Annual Standard Deviation for ARKK: {annual_std_arkk * 100:.3f}")

Average Annual Return for ARKK: 9.320
Average Annual Excess Return for ARKK: 7.886
Annual Standard Deviation for ARKK: 41.724


### Problem 6.b

In [10]:
# Compute the annualized Sharpe ratio for ARKK and Mkt-Rf
sharpe_arkk = (df_merge["Excess Return_ARKK"].mean() / df_merge["CtoC_Adj"].std()) * np.sqrt(12)

# Mkt-Rf Sharpe
sharpe_mkt = (df_merge["Mkt-RF"].mean() / df_merge["Mkt-RF"].std()) * np.sqrt(12)

# Display results
print(f"Annualized Sharpe Ratio for ARKK: {sharpe_arkk:.3f}")
print(f"Annualized Sharpe Ratio for Mkt-Rf: {sharpe_mkt:.3f}")
# Evaluate if ARKK has a higher Sharpe ratio than Mkt-Rf
if sharpe_arkk > sharpe_mkt:
    print("ARKK has a higher Sharpe ratio than benchmark Mkt-Rf.")
else:
    print("ARKK does not have a higher Sharpe ratio than benchmark Mkt-Rf.")

Annualized Sharpe Ratio for ARKK: 0.189
Annualized Sharpe Ratio for Mkt-Rf: 0.582
ARKK does not have a higher Sharpe ratio than benchmark Mkt-Rf.


### Problem 6.c

In [11]:
# Do the CAPM regression of Excess Return_ARKK on Mkt-RF
import statsmodels.api as sm
df_merge['Mkt-RF'] = df_merge['Mkt-RF'] / 100
X = sm.add_constant(df_merge['Mkt-RF'])
y = df_merge['Excess Return_ARKK']
capm_model = sm.OLS(y, X).fit()
# Extract alpha, beta, and their t-statistics
alpha = capm_model.params['const']
beta = capm_model.params['Mkt-RF']
# Compute annualized alpha
annualized_alpha = alpha * 12
# Display results
print(f"Beta: {beta:.3f}, t-statistic: {capm_model.tvalues['Mkt-RF']:.3f}")
print(f"Annualized Alpha: {annualized_alpha * 100:.3f}")

Beta: 1.675, t-statistic: 9.598
Annualized Alpha: -11.127


### Problem 6.d

In [12]:
# Do the Fama-French 3-factor regression of Excess Return_ARKK on Mkt-RF, SMB, and HML
df_merge['SMB'] = df_merge['SMB'] / 100
df_merge['HML'] = df_merge['HML'] / 100
X_ff3 = sm.add_constant(df_merge[['Mkt-RF', 'SMB', 'HML']])
ff3_model = sm.OLS(y, X_ff3).fit()
# Extract alpha, beta, and their t-statistics
alpha_ff3 = ff3_model.params['const']
beta_mkt = ff3_model.params['Mkt-RF']
beta_smb = ff3_model.params['SMB']
beta_hml = ff3_model.params['HML']
# Compute annualized alpha
annualized_alpha_ff3 = alpha_ff3 * 12
# Display results
print(f"Factor Loadings:")
print(f"(Beta (Mkt-RF): {beta_mkt:.3f}, Beta (SMB): {beta_smb:.3f}, Beta (HML): {beta_hml:.3f})")
print(f"Annualized Alpha (FF3): {annualized_alpha_ff3 * 100:.3f}")

Factor Loadings:
(Beta (Mkt-RF): 1.440, Beta (SMB): 1.535, Beta (HML): -0.835)
Annualized Alpha (FF3): -7.372


## <mark> Problem solution summary: <mark>

From the above questions in **problem 6**, we load the ARKK and Fama-French factor data from the CSV files into pandas dataframes. We filter the ARKK data to include only the months from June 2018 to June 2023 and compute the monthly close-to-close simple returns using the adjusted close prices. We then merge the ARKK returns with the Fama-French factors on the Date index. We calculate the excess returns for ARKK by subtracting the risk-free rate (RF) from the monthly returns. We compute the average annual return, average annual excess return, and annual standard deviation for ARKK. Next, we calculate the annualized Sharpe ratio for both ARKK and the market factor (Mkt-RF) to evaluate ARKK's performance relative to the benchmark. We perform a CAPM regression of ARKK's excess returns on Mkt-RF to obtain the beta and annualized alpha. Finally, we conduct a Fama-French 3-factor regression of ARKK's excess returns on Mkt-RF, SMB, and HML to extract the factor loadings and annualized alpha, providing insights into ARKK's investment strategy and performance.

### Problem 6.e

The CAPM alpha and the Fama-French 3-factor alpha differ because the CAPM model only accounts for exposure to the overall market factor (Mkt-Rf).  All idiosyncratic return component is captured as unexplained abnormal return (alpha) in CAPM. In contrast, the Fama-French 3-factor model incorporates SMB (size) and HML (value) factors, which explain additional sources of variation in stock returns. As a result, the absolute value of alpha under the 3-factor model is typically smaller than the absolute value of CAPM alpha, since part of what CAPM attributes to "skill" or abnormal performance is explained away by systematic exposure to size or value factors.

The factor loadings from the 3-factor regression provide insights into stocks which ARKK tends to hold. A positive and significant loading on SMB indicates that ARKK has a preference for small-cap or growth-oriented firms rather than large-cap firms. A negative loading on HML suggests that ARKK invests more in growth stocks as opposed to value stocks. Combined with a high beta with respect to the market factor (Mkt-Rf), these results are consistent with ARKK’s investment strategy, which focuses on innovative, high-growth and smaller-cap companies.